In [2]:
import pyspark
sc = pyspark.SparkContext()

In [3]:
import sqlite3
import datetime
import time
from time import strptime

from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import RandomForest

bd = sqlite3.connect('velib3.db')

c = bd.cursor()
c.execute("select DISTINCT num from station_dyn" )

list_station=[]

for row in c:
    list_station.append(row[0]) 

"""
for h in range(0,24):
    for m in range(0,51,10):
        #print ("0" if len(str(h))==1 else "")+str(h)+":"+str(m)+("0" if m==0 else "") #144 intervalles par jour
        pass"""

data=[]
interval_pred=600*3


for station in range(15003,15004):#0,1):#len(list_station)):
    c.execute("select nb_velo_dispo, last_update, last_update_date from station_dyn where num = "+str(15003)+" order by last_update")
    #c.execute("select nb_velo_dispo, last_update, last_update_date from station_dyn where num = "+str(list_station[station])+" order by last_update")
    
    list_row=c.fetchall()
    
    """i=0
    while list_row!=[]:
        print list_row.pop(0)
        i+=1
        print str(i)+(" vide" if list_row==[] else " non vide")
    print len(list_row)
    break"""
    
    #print list_row[-1]
    #print time.strftime("%d/%m/%Y-%H:%M:%S", time.gmtime((int(list_row[-1][1])/1000/600)*600))
    #print time.strftime("%d/%m/%Y-%H:%M:%S", time.gmtime((int(list_row[0][1])/1000/600)*600+600))
    
    row=1
    boucle=0
    in_flow=0
    out_flow=0
    date = (int(list_row[0][1])/1000/interval_pred)*interval_pred+interval_pred
    #for date in range((int(list_row[0][1])/1000/600)*600+600,(int(list_row[-1][1])/1000/600)*600+1,600):
        #print datetime.datetime.fromtimestamp(date).strftime('%H:%M'), datetime.datetime.fromtimestamp(date).strftime('%d-%m')
        
        #implémenter les filtres
    prec_row=list_row[0]
    current_row=list_row.pop(0) #Erreur si la liste est vide
    while list_row!=[]:
        current_row=list_row.pop(0)
        row+=1
        
        flow=current_row[0]-prec_row[0]


        
        if int(current_row[1])/1000<date:
            boucle+=1
            in_flow += flow if flow>0 else 0
            out_flow += flow if flow<0 else 0
        else: #in_flow : nb_velo_dispo, week-day, heure_normalisée
            data.append(LabeledPoint(float(in_flow),[float(current_row[0]),time.gmtime(int(current_row[1])/1000)[6],int(time.strftime("%H", time.gmtime(date+3600-interval_pred)))*6+int(time.strftime("%M", time.gmtime(date+3600-interval_pred)))/10]))
            print [LabeledPoint(float(in_flow),\
                   [current_row[0],time.gmtime(int(current_row[1])/1000)[6],\
                   int(time.strftime("%H", time.gmtime(date+3600-interval_pred)))*6+int(time.strftime("%M", time.gmtime(date+3600-interval_pred)))/10])]
            while int(current_row[1])/1000>=date:
                date+=interval_pred
            boucle=1
            in_flow = flow if flow>0 else 0
            out_flow = flow if flow<0 else 0
        
        print boucle, row, current_row, time.strftime("%H:%M:%S", time.gmtime((int(current_row[1])/1000+3600)-(date+3600-interval_pred)))
        print "intervalle : ["+str(time.strftime("%H:%M", time.gmtime(date+3600-interval_pred)))+","+str(time.strftime("%H:%M", time.gmtime(date+3600)))+"["+" -  n° : "+str(int(time.strftime("%H", time.gmtime(date+3600-interval_pred)))*2+int(time.strftime("%M", time.gmtime(date+3600-interval_pred)))/30) #doit être modifié à la main si changemenr d'intervalle
        
        print "in flow : "+str(in_flow)+" - out flow : "+str(out_flow)
        
        prec_row=current_row




c.close()

1 2 (0, u'1489453088000', u'2017-03-14 01:58:08') 00:28:08
intervalle : [01:30,02:00[ -  n° : 3
in flow : 0 - out flow : 0
[LabeledPoint(0.0, [0.0,1.0,9.0])]
1 3 (0, u'1489453692000', u'2017-03-14 02:08:12') 00:08:12
intervalle : [02:00,02:30[ -  n° : 4
in flow : 0 - out flow : 0
2 4 (0, u'1489454295000', u'2017-03-14 02:18:15') 00:18:15
intervalle : [02:00,02:30[ -  n° : 4
in flow : 0 - out flow : 0
3 5 (0, u'1489454899000', u'2017-03-14 02:28:19') 00:28:19
intervalle : [02:00,02:30[ -  n° : 4
in flow : 0 - out flow : 0
[LabeledPoint(0.0, [0.0,1.0,12.0])]
1 6 (0, u'1489455502000', u'2017-03-14 02:38:22') 00:08:22
intervalle : [02:30,03:00[ -  n° : 5
in flow : 0 - out flow : 0
2 7 (0, u'1489456106000', u'2017-03-14 02:48:26') 00:18:26
intervalle : [02:30,03:00[ -  n° : 5
in flow : 0 - out flow : 0
3 8 (0, u'1489456710000', u'2017-03-14 02:58:30') 00:28:30
intervalle : [02:30,03:00[ -  n° : 5
in flow : 0 - out flow : 0
[LabeledPoint(0.0, [0.0,1.0,15.0])]
1 9 (0, u'1489457297000', u'2017

In [4]:
print len(data)
print "********"
print data

794
********
[LabeledPoint(0.0, [0.0,1.0,9.0]), LabeledPoint(0.0, [0.0,1.0,12.0]), LabeledPoint(0.0, [0.0,1.0,15.0]), LabeledPoint(0.0, [0.0,1.0,18.0]), LabeledPoint(0.0, [0.0,1.0,21.0]), LabeledPoint(0.0, [0.0,1.0,24.0]), LabeledPoint(0.0, [0.0,1.0,27.0]), LabeledPoint(0.0, [1.0,1.0,30.0]), LabeledPoint(2.0, [2.0,1.0,33.0]), LabeledPoint(1.0, [3.0,1.0,36.0]), LabeledPoint(1.0, [4.0,1.0,39.0]), LabeledPoint(5.0, [10.0,1.0,42.0]), LabeledPoint(9.0, [16.0,1.0,45.0]), LabeledPoint(8.0, [12.0,1.0,48.0]), LabeledPoint(8.0, [0.0,1.0,51.0]), LabeledPoint(9.0, [6.0,1.0,54.0]), LabeledPoint(13.0, [16.0,1.0,57.0]), LabeledPoint(1.0, [10.0,1.0,60.0]), LabeledPoint(2.0, [9.0,1.0,63.0]), LabeledPoint(2.0, [8.0,1.0,66.0]), LabeledPoint(1.0, [5.0,1.0,69.0]), LabeledPoint(7.0, [8.0,1.0,72.0]), LabeledPoint(2.0, [0.0,1.0,75.0]), LabeledPoint(1.0, [0.0,1.0,78.0]), LabeledPoint(4.0, [3.0,1.0,81.0]), LabeledPoint(7.0, [4.0,1.0,84.0]), LabeledPoint(1.0, [5.0,1.0,87.0]), LabeledPoint(0.0, [3.0,1.0,90.0]), L

In [5]:
(training, test) = sc.parallelize(data).randomSplit([0.7, 0.3])

training.collect()

#model = RandomForest.trainClassifier(training, numClasses=8, categoricalFeaturesInfo={}, numTrees=100, impurity='gini')
model =  RandomForest.trainClassifier(training, 25, categoricalFeaturesInfo={},\
numTrees=100, featureSubsetStrategy='auto', impurity='gini', maxDepth=5, maxBins=32, seed=None)


#print(model.toDebugString())

from pyspark.mllib.evaluation import MulticlassMetrics

def getPredictionsLabels(model, test_data):
    predictions = model.predict(test_data.map(lambda r: r.features))
    return predictions.zip(test_data.map(lambda r: r.label))

def printMetrics(predictions_and_labels):
    metrics = MulticlassMetrics(predictions_and_labels)
    print 'Precision of True ', metrics.precision(1)
    print 'Precision of False', metrics.precision(0)
    print 'Recall of True    ', metrics.recall(1)
    print 'Recall of False   ', metrics.recall(0)
    print 'F-1 Score         ', metrics.fMeasure()
    print 'Confusion Matrix\n', metrics.confusionMatrix().toArray()
    
predictions_and_labels = getPredictionsLabels(model, test)

printMetrics(predictions_and_labels)

prediction = model.predict(test.map(lambda x: x.features))
labeled_pred = test.map(lambda lp: lp.label).zip(prediction)

testErr = labeled_pred.filter(lambda (v, p): v != p).count()/float(test.count())
print('Test Error = ' + str(testErr))
print('Learned classification forest model')


Precision of True  0.279569892473
Precision of False 0.686567164179
Recall of True     0.464285714286
Recall of False    0.707692307692
F-1 Score          0.38
Confusion Matrix
[[ 46.  14.   3.   0.   1.   0.   1.   0.   0.   0.   0.   0.   0.   0.
    0.   0.]
 [ 17.  26.   6.   0.   7.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.]
 [  1.  15.   7.   0.   4.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.]
 [  3.  15.   1.   0.   4.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.]
 [  0.   7.   2.   0.  12.   0.   1.   0.   0.   0.   0.   0.   0.   0.
    0.   0.]
 [  0.   2.   2.   0.   4.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.]
 [  0.   4.   2.   0.   1.   0.   1.   0.   0.   0.   0.   0.   0.   0.
    0.   0.]
 [  0.   3.   1.   1.   3.   0.   1.   0.   1.   0.   0.   0.   0.   0.
    0.   0.]
 [  0.   2.   3.   0.   1.   0.   0.   0.   3.   0.   0.   0.   0.   0.
    0.   0.]
 [  0.   2.   0.   0.   2.   0.   0.   0.   1.   0.   0.  